In [1]:
from dl_util import *
from ml_util import *
from run_eval import split_fit_plot_predict
from eval_class_util import *

Using TensorFlow backend.


### Loading Dataset

In [2]:
df = pd.read_csv('eval_datasets/HIV.csv')
hiv = df[pd.notnull(df['HIV_active'])]

In [3]:
hiv.head()

,smiles,activity,HIV_active
0,CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)...,CI,0
1,C(=Cc1ccccc1)C1=[O+][Cu-3]2([O+]=C(C=Cc3ccccc3...,CI,0
2,CC(=O)N1c2ccccc2Sc2c1ccc1ccccc21,CI,0
3,Nc1ccc(C=Cc2ccc(N)cc2S(=O)(=O)O)c(S(=O)(=O)O)c1,CI,0
4,O=S(=O)(O)CCS(=O)(=O)O,CI,0


In [4]:
hiv_true = hiv[hiv['HIV_active']==1]

In [5]:
hiv_false_selected = hiv[hiv['HIV_active']== 0][:1443]

In [6]:
hiv_smiles = hiv_true['smiles'].tolist()+hiv_false_selected['smiles'].tolist()
hiv_active = hiv_true['HIV_active'].tolist()+hiv_false_selected['HIV_active'].tolist()
hiv_indices = np.concatenate((hiv_true.index.values,hiv_false_selected.index.values))

In [7]:
hiv_maccs_all = loadNumpy('maccs_hiv_all')
hiv_maccs = []
for i in hiv_indices:
    hiv_maccs += [hiv_maccs_all[i]]
hiv_maccs = np.array(hiv_maccs)

In [8]:
hiv['HIV_active'].describe()

count    41127.000000
mean         0.035086
std          0.184001
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: HIV_active, dtype: float64

In [9]:
len(hiv_smiles), len(hiv_active), len(hiv_indices)

(2886, 2886, 2886)

In [10]:
saveData(hiv_smiles, 'hiv_smiles_2886')

### Data Preparation

In [13]:
len_hiv_smiles = []
for SMILE in hiv_smiles:
    len_hiv_smiles += [len(SMILE)]
    
max_len_hiv = max(len_hiv_smiles)

hiv_vocab_size = generate_onehot_encoding(hiv_smiles,'vocab_size')
hiv_vocab_size, max_len_hiv

(54, 400)

In [14]:
hiv_encodings = generate_onehot_encoding(hiv_smiles)
hiv_sequences = sequence.pad_sequences(hiv_encodings, maxlen=max_len_hiv)

In [21]:
X1 = hiv_sequences
X2 = hiv_maccs 
Y =  hiv_active
# X1_train_hiv, X1_test_hiv, y_train_hiv, y_test_hiv = train_test_split(X1_hiv, Y_hiv, random_state=1024)
# X2_train_hiv, X2_test_hiv, y_train_hiv, y_test_hiv = train_test_split(X2_hiv, Y_hiv, random_state=1024)
saveNumpy(hiv_sequences, 'hiv_sequences')
saveNumpy(hiv_maccs, 'hiv_maccs' )
saveNumpy(hiv_active, 'hiv_active')

In [2]:
hiv_maccs = loadNumpy('hiv_maccs')
hiv_sequences = loadNumpy('hiv_sequences')
hiv_active = loadNumpy('hiv_active')

In [3]:
hiv_vocab_size, max_len_hiv = 54,400

In [6]:
split_fit_plot_predict(cnn_model,  hiv_sequences, hiv_maccs, hiv_active, hiv_vocab_size, max_len_hiv, "hiv", batch_size=4,epochs=5)

cnn_model
classification
++++++++++++++
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 32)           1728      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 400, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 200, 32)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 200, 32)           3104      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 100, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3200)              0         
___________________

KeyboardInterrupt: 

In [68]:
model = mlp_model()
history = model.fit(X2_train_hiv, y_train_hiv, shuffle=True, validation_split=0.2, \
                        epochs=20, batch_size=128, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 512)               86016     
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_14 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 65        
_________________________________________________________________
dense_41 (Dense)             (None, 1)                 2         
Total para

In [24]:
model = cnn_rnn_model(vocab=hiv_vocab_size,max_len=max_len_hiv)
history = model.fit(X1_train_hiv, y_train_hiv, shuffle=True, validation_split=0.2, \
                        epochs=20, batch_size=128, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 400, 32)           1728      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 400, 32)           3104      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 200, 32)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 58,133
Trainable params: 58,133
Non-trainable params: 0
_________________________________________________________________
Train on 1731 samples, validate on 433 samples
Epoch 1/20
1731/1731 [==============================] - 4s 2ms/step - loss: 0.6900 - precis

In [59]:
model = merged_model(vocab=hiv_vocab_size,max_len=max_len_hiv)
history = model.fit([X1_train_hiv,X2_train_hiv], y_train_hiv, shuffle=True, validation_split=0.2, \
                        epochs=20, batch_size=128, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_6 (Merge)              (None, 128)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_12 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 65        
Total params: 271,777
Trainable params: 271,777
Non-trainable params: 0
_________________________________________________________________
Train on 1731 samples, validate on 433 samples
Epoch 1/20
1731/1731 [==============================] - 6s 3ms/step - loss: 0.5692 - precision: 0.4479 - recall: 0.9261 - auc: 0.6696 - val_loss: 0.5185 - val_precision: 0.4960 - val_recall: 1.0000 - val_auc: 0.7879
Epoch

In [221]:
model = cnn_model(vocab=hiv_vocab_size,max_len=max_len_hiv)
history = model.fit(X1_train_hiv, y_train_hiv, shuffle=True, validation_split=0.2, \
                        epochs=20, batch_size=128, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_42 (Embedding)     (None, 400, 32)           1728      
_________________________________________________________________
conv1d_66 (Conv1D)           (None, 400, 32)           3104      
_________________________________________________________________
max_pooling1d_66 (MaxPooling (None, 200, 32)           0         
_________________________________________________________________
conv1d_67 (Conv1D)           (None, 200, 32)           3104      
_________________________________________________________________
max_pooling1d_67 (MaxPooling (None, 100, 32)           0         
_________________________________________________________________
flatten_33 (Flatten)         (None, 3200)              0         
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 3201      
Total para

In [170]:

model.evaluate(X1_test_hiv,y_test_hiv)[1]

10282/10282 [==============================] - 1s 99us/step


0.9678078194903715

In [222]:
y_prob = model.predict(X1_test_hiv)
y_predict_hiv = y_prob.argmax(axis=-1)
auc(y_predict_hiv, y_test_hiv)

AttributeError: 'list' object has no attribute 'dtype'